In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

# Download Apple stock data
aapl = yf.download('AAPL', start='2010-01-01', end='2023-10-31')
# Clean the data
aapl_cleaned = aapl.dropna()

# Perform feature engineering
aapl_features = pd.DataFrame()
aapl_features['Close'] = aapl_cleaned['Close']
aapl_features['MA_50'] = aapl_cleaned['Close'].rolling(window=50).mean()
aapl_features['MA_200'] = aapl_cleaned['Close'].rolling(window=200).mean()

# Normalize the data
aapl_normalized = (aapl_features - aapl_features.min()) / (aapl_features.max() - aapl_features.min())


[*********************100%%**********************]  1 of 1 completed


In [3]:
# Reshape data for VGG16 input
train_data = aapl_normalized.values.reshape((aapl_normalized.shape[0], aapl_normalized.shape[1], 1))

# Split the data into train and test sets
train_size = int(len(aapl_normalized) * 0.8)
train_data = aapl_normalized[:train_size]
test_data = aapl_normalized[train_size:]

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(train_data.shape[1], train_data.shape[2], 1))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add a custom classifier on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)


IndexError: tuple index out of range